In [1]:
import numpy as np
import xarray as xr
import datetime as dt
import os
import matplotlib.pyplot as plt

In [2]:
is_detrend= 1#0 #1 to use detrended anomalies (default), 0 otherwise

In [3]:
# Years to process and years to use as climatology for thresholds
years = [1991, 2020]
clim_years = [1991, 2020]

basepath='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW'
# Input/output directory
if is_detrend:
    dir = basepath+'/mhw/detrended';
else:
    dir = basepath+'/mhw';

# Model names
#           1              2               3            4              5             6
mods = ['CanCM4i', 'COLA-RSMAS-CCSM4', 'GEM-NEMO', 'GFDL-SPEAR', 'NASA-GEOSS2S', 'NCEP-CFSv2']#
nmod = len(mods)
nl = dict(zip(mods,[11, 11, 11, 11, 8, 9])) # Max lead time for each model
#mods = [ 'GFDL-SPEAR', 'NASA-GEOSS2S', 'NCEP-CFSv2']#

In [ ]:
# Loop through lead times
print('\nBuilding multimodel ensemble for NMME MHW forecasts...\n')
print('Lead')
for il in (8,):#range(0,np.max(list(nl.values()))):
    print(' ',il)
        
    # Loop through models
    is_mhw_ens=[]
    mlist=[]
    flist=[]
    for modi in mods:
    
        if il<nl[modi]:
            
            # Load MHWs
            if is_detrend:
                f_in = f'{dir}/mhw_{modi}_l{il}_detrended_{years[0]}_{years[1]}.nc'
            else:
                f_in = f'{dir}/mhw_{modi}_l{il}_{years[0]}_{years[1]}.nc'
                print(f_in)
            flist.append(f_in)
    fin=xr.open_mfdataset(flist,chunks={'X':10,'Y':10,'M':-1},concat_dim='M',combine='nested',data_vars='minimal',
                       coords='minimal',parallel=True,preprocess=lambda f: f.drop_vars(["sst_an_thr","mhw_prob"]) )
    fin2=fin.chunk({'X':10,'Y':10,'M':-1})
    out=fin2.mean(dim='M').rename({'is_mhw':'mhw_prob'})
    # Save to file
    if is_detrend:
        f_out = f'{dir}/mhw_MME_l{il}_detrended_{years[0]}_{years[1]}.nc'
        print(f_out)
    else:
        f_out = f'{dir}/mhw_MME_l{il}_{years[0]}_{years[1]}.nc'
    out.to_netcdf(f_out,mode='w')
    fin.close()
#             fin=xr.open_dataset(f_in)
            
#             is_mhw_ens.append(fin.is_mhw.values)
#             # Record which model is associated with which forecasts
#             mlist.append(modi)
#             fin.close()

#     # concatenate
#     is_mhw_ens=np.concatenate(is_mhw_ens,axis=1) # concatenate along ensemble member axis
#     # Find forecast MHW probability
#     mhw_prob = np.squeeze(np.nanmean(is_mhw_ens,axis=1))


#     xout=xr.Dataset(data_vars={'lon':(['X',],fin.lon.values),
#                             'lat':(['Y',],fin.lat.values),
#                             'time':(['S'],fin.time.values),
#                             'year':(['S'],fin.year.values),
#                             'month':(['S'],fin.month.values),
#                             'is_mhw':(['S','M','Y','X'],is_mhw_ens),
#                             'mhw_prob':(['S','Y','X'],mhw_prob)},
#                 coords=dict(X=fin.X,Y=fin.Y,M=fin.M,S=fin.S),)
#     xout.to_netcdf(f_out,mode='w')
#     del is_mhw; del is_mhw_ens; del mhw_prob;
print('\nDone\n\n')


Building multimodel ensemble for NMME MHW forecasts...

Lead
  8
/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW/mhw/detrended/mhw_MME_l8_detrended_1991_2020.nc


In [ ]:
break

In [ ]:
flist=['/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW/mhw/mhw_CanCM4i_l0_1991_2020.nc',
       '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW/mhw/mhw_COLA-RSMAS-CCSM4_l0_1991_2020.nc',
      '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW/mhw/mhw_GEM-NEMO_l0_1991_2020.nc',
       '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW/mhw/mhw_GFDL-SPEAR_l0_1991_2020.nc',
       '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW/mhw/mhw_NASA-GEOSS2S_l0_1991_2020.nc',
       '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW/mhw/mhw_NCEP-CFSv2_l0_1991_2020.nc']

In [ ]:
test=xr.open_mfdataset(flist,chunks={'X':10,'Y':10,'M':-1},concat_dim='M',combine='nested',data_vars='minimal',
                       coords='minimal',parallel=True,preprocess=lambda f: f.drop_vars(["sst_an_thr","mhw_prob"]) )

In [ ]:
test2=test.chunk({'X':10,'Y':10,'M':-1})
test2.is_mhw

In [ ]:
out=test.mean(dim='M')

In [ ]:
out

In [ ]:
plt.plot(out.is_mhw[:,140,200])